In [5]:
import numpy as np, pandas as pd
import glob
import os 
import json
import sys
import numpy
from math import log, log10
from gdpyc import GasMap, DustMap
from astropy.coordinates import SkyCoord
import matplotlib
from matplotlib import pyplot as plt

df = pd.read_csv('./FGL_11152023_GammaCan.csv')

df = df[df['FGL'].str[-1]!='c']

df = df.sort_values(by=['flux_aper90_ave_b'], ascending=False).reset_index(drop=True)

# print(df.columns)

df[['FGL','X_name','Class','name','flux_aper90_ave_b']]

df['name_nospace'] = df['X_name'].str.replace(' ','')#apply(lambda x: x.replace(' ', ''))



df_alls = pd.read_csv('./FGL_11152023_GammaCan_alls.csv')

# light curve >50 counts
# spectrum > 100 counts

df_select3 = df[df['X_name'].isin(df_alls.loc[df_alls['src_cnts_aper90_b']>=50, 'name'].unique())].reset_index(drop=True)

df_select3 = df_select3[~df_select3['X_name'].isin(['2CXO J174457.8-290509', '2CXO J174507.0-290357','2CXO J174506.8-290537','2CXO J111459.1-611707','2CXO J150234.5+015205','2CXO J073717.0+653557'])].reset_index(drop=True)
# print(df_select3)

coords = SkyCoord(df_select3['ra'], df_select3['dec'], unit='deg')
df_select3['ebv'] = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
df_select3['nH_from_AV'] = 2.21 * 3.1 * df_select3['ebv'] * 0.1
df_select3['nH']  = np.log10(GasMap.nh(coords, nhmap='LAB').value) #/ 1e22 # nH in unit of 1.e22 atoms /cm2

print(df_select3[['FGL','X_name','Class','CT','name','name_nospace','ra', 'dec','flux_aper90_ave_b','var_intra_prob',
       'var_inter_prob','nH']])

             FGL                 X_name    Class         CT  \
0   J0859.3-4342  2CXO J085927.0-434528       CV   2.451807   
1   J1725.1-3408  2CXO J172508.8-341112      YSO   0.335596   
2   J0859.2-4729  2CXO J085905.6-473041      YSO   4.286869   
3   J1742.5-2833  2CXO J174216.9-283707       CV   1.907402   
4   J1743.8-3143  2CXO J174347.2-314025  LM-STAR   4.563578   
..           ...                    ...      ...        ...   
56  J1510.9+0551  2CXO J151100.4+054921      AGN   3.911411   
57  J0859.3-4342  2CXO J085928.4-434628       NS   2.893635   
58  J1843.7-3227  2CXO J184332.8-322122      AGN   2.564143   
59  J1510.9+0551  2CXO J151051.3+054937      AGN  41.413040   
60  J1242.6+3236  2CXO J124235.1+323340      AGN  24.799075   

                       name          name_nospace          ra        dec  \
0   2CXO J085927.0-434528-1  2CXOJ085927.0-434528  134.862609 -43.757805   
1   2CXO J172508.8-341112-1  2CXOJ172508.8-341112  261.286915 -34.186814   
2   2CXO J08590

In [10]:
for index, df_src in df_select3.iterrows():# ['name_nospace'].values:
    
    # if index >=2:
    #     continue

    src = df_src['name_nospace']
    
    dir_abs = f'/home/orion51/Desktop/Research/spectral_fitting/FGL_spectral_fitting/Individual_sources/{src}/'

    os.system(f'mkdir /home/orion51/Desktop/Research/spectral_fitting/FGL_spectral_fitting/Individual_sources/bxa_results/{src}')
    for md, md_f in zip(['pl', 'mk', 'bb'], ['powerlaw','mekal','bb']):
        os.system(f'cp -rf {dir_abs}{md_f}_out_ /home/orion51/Desktop/Research/spectral_fitting/FGL_spectral_fitting/Individual_sources/bxa_results/{src}/')

mkdir: cannot create directory ‘/home/orion51/Desktop/Research/spectral_fitting/FGL_spectral_fitting/Individual_sources/bxa_results/2CXOJ085927.0-434528’: File exists
mkdir: cannot create directory ‘/home/orion51/Desktop/Research/spectral_fitting/FGL_spectral_fitting/Individual_sources/bxa_results/2CXOJ172508.8-341112’: File exists


In [7]:


for index, df_src in df_select3.iterrows():# ['name_nospace'].values:
    
    src = df_src['name_nospace']
    src_short = src[4:]

    # if index >=2:
    #     continue

    # df_src = df_select[df_select['name_nospace']==src]
    ra, dec, src_name = df_src['ra'], df_src['dec'], df_src['X_name']
    print(src_name, ra, dec)
    
    src_stat = {'name': src_short, 'Class':df_src['Class'], 'CT':df_src['CT'], 'nH_LAB':df_src['nH']}


    # df_var = search_file.table.to_pandas()
    df_var = df_alls[(df_alls['name']==src_name) & (df_alls['src_cnts_aper90_b']>=50)].reset_index(drop=True)
    # print(df_var[['name','obsid','obi','region_id','src_cnts_aper90_b']])
    # print(df_var['powlaw_nh'][0]/100,nH_from_AV,nH)
    
    # '''
    df_var['obsid'] = df_var['obsid'].apply(lambda x: str(x).zfill(5))

    src_stat['nobs'] = len(df_var)
    src_stat['ncounts'] = df_var['src_cnts_aper90_b'].sum()
    src_stat['fb'] = df_src['flux_aper90_ave_b']
    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md twomekal')
    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md powerlawbb')

    dir_abs = f'/home/orion51/Desktop/Research/spectral_fitting/FGL_spectral_fitting/Individual_sources/{src}/'

    for md, md_f in zip(['pl', 'mk', 'bb'], ['powerlaw','mekal','bb']):
        
        fig = plt.figure(figsize=(6, 4))
        gs = fig.add_gridspec(2, 1, hspace=0.05,height_ratios=[3, 1])
        ax1 = fig.add_subplot(gs[0])
        ax2 = fig.add_subplot(gs[1], sharex=ax1)

        for idx, color in zip( range(len(df_var)), ['steelblue','green', 'red', 'meganta']):
            
            row = df_var.iloc[[idx]]
            obsid, obi, region_id, counts = row['obsid'], row['obi'], row['region_id'], row['src_cnts_aper90_b']
            
            df = pd.read_csv(f'{dir_abs}{md_f}_out_/src_subtract_{idx+1}.txt.gz',sep=" ", header=None, names=['dx','dxr','dy','dyr','mx','mylo','myid','myhi'])  
  

            plt.sca(ax1)
            
            
            ax1.errorbar(df['dx'], df['dy'], xerr=df['dxr']/2, yerr=df['dyr'], fmt='.', alpha=0.7, label=f'ObsID={row["obsid"].values[0]}, net_cnts={row["src_cnts_aper90_b"].values[0]:.1f}', color=color)
            
            # if idx ==len(df_var)-1:
            xs = df['dx'].values-df['dxr'].values/2
            ys = df['myid'].values
            ax1.step(x=np.append(xs, 10), y=np.append(ys, ys[-1]), color=color,where='post',alpha=1)
            
            plt.xlim([0.45,8])
            plt.xscale('log')
            ax1.set_yscale('log')
            plt.legend()
            plt.gcf().axes[0].set_title(f'{src} {md_f}')
            

            
            plt.sca(ax2)
            # exactly replicates plot_ratio
            plt.errorbar(x=df['dx'], y=df['dy']/df['myid'], xerr=df['dxr']/2, yerr=df['dyr']/df['myid'], fmt='.', color=color)
            plt.yscale('log')

        ax1.set_ylabel('Counts/s/keV')
        ax2.get_xaxis().set_minor_formatter(matplotlib.ticker.ScalarFormatter())
        ax2.set_xticks([0.5, 1, 2, 3, 5, 6, 7], minor=True)
        ax2.set_xlabel('Energy (keV)')
        ax2.set_ylabel('Data/Model')
            
        ax2.plot([0.4, 9], [1, 1], color='k')
        plt.savefig(f'./html/images/{src}_{md}_spectrum.png', dpi=300)
        plt.close()

        os.system(f'cp {dir_abs}{md_f}_out_/plots/corner.pdf ./html/images/{src}_{md}_corner.pdf')

            

    

2CXO J085927.0-434528 134.86260894620565 -43.75780494871906
2CXO J172508.8-341112 261.2869153895182 -34.18681384796624
2CXO J085905.6-473041 134.7735453758357 -47.51135832293674
2CXO J174216.9-283707 265.570522913482 -28.61890444484742
2CXO J174347.2-314025 265.94681015215986 -31.6736519992628
2CXO J085932.2-434602 134.88419226775284 -43.76731050729919
2CXO J063805.1-801854 99.52142687586397 -80.31491311531052
2CXO J085928.0-434510 134.86702561226645 -43.75301883822596
2CXO J085926.2-434527 134.85927561432112 -43.757721614919234
2CXO J005806.2-460419 14.525923779713466 -46.0719377120392
2CXO J174042.0-280724 265.1749809894933 -28.123598413371127
2CXO J085902.1-473016 134.75896206054904 -47.50462499008288
2CXO J085903.2-434834 134.76344230816275 -43.80951604600068
2CXO J174036.5-280840 265.15204415925945 -28.144598705407148
2CXO J174237.6-283726 265.65692088726325 -28.624160710801192
2CXO J085910.9-434343 134.79544231604427 -43.72886327267862
2CXO J033532.7-072741 53.88645833333332 -7.4

/home/orion51/miniconda3/envs/ciao-4.15.2/lib/python3.10/site-packages/matplotlib/axes/_axes.py:3680: RuntimeWarning: invalid value encountered in add
  low, high = dep + np.vstack([-(1 - lolims), 1 - uplims]) * err
/home/orion51/miniconda3/envs/ciao-4.15.2/lib/python3.10/site-packages/matplotlib/axes/_axes.py:3680: RuntimeWarning: invalid value encountered in add
  low, high = dep + np.vstack([-(1 - lolims), 1 - uplims]) * err
/home/orion51/miniconda3/envs/ciao-4.15.2/lib/python3.10/site-packages/matplotlib/axes/_axes.py:3680: RuntimeWarning: invalid value encountered in add
  low, high = dep + np.vstack([-(1 - lolims), 1 - uplims]) * err


2CXO J063623.7-801259 99.09885007105343 -80.2164600981167
2CXO J064013.0-801724 100.0539632799206 -80.28998842512243
2CXO J143550.3+333338 218.95984778296227 33.56057620836909
2CXO J110823.6+091239 167.09866666666667 9.211041666666665
2CXO J103302.5+572833 158.26040411770464 57.47597910222971
2CXO J103224.8+573153 158.10371203349163 57.53155199799183
2CXO J103225.1+572814 158.10465341039065 57.47067149211448
2CXO J103142.0+573015 157.92511180087524 57.5043993102671
2CXO J161945.8-504222 244.9408733190159 -50.7062801642946
2CXO J125751.0+273231 194.4624654400349 27.541988900949683
2CXO J125734.0+272730 194.39159657415053 27.458358894092232
2CXO J125745.0+273210 194.4378404633184 27.53610277635087
2CXO J151100.5+054912 227.7520571001258 5.82012292504521
2CXO J103145.7+573401 157.9407785139211 57.5671076400644
2CXO J103143.3+573157 157.93061182194657 57.532560420134814
2CXO J044308.1+361310 70.78404166666667 36.21944722222222
2CXO J181908.5-204037 274.78553483422075 -20.676901095677213
2C